In [1]:
import requests
import folium
import json
import pystac
import rioxarray
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl
import fsspec
import geopandas as gpd
import branca
from branca.colormap import LinearColormap
from folium import plugins
import adlfs

In [2]:
# use read only token
sas_token = "se=2023-02-27T21%3A29Z&sp=racwdl&sv=2018-11-09&sr=c&skoid=b4d39ccb-ca9c-4e9d-a183-5e0f3ba3e609&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-02-20T21%3A30%3A01Z&ske=2023-02-27T21%3A29%3A00Z&sks=b&skv=2018-11-09&sig=p0mG2aPIjQaBOhBFGGf%2BnGJLZhsarSAn2zoWfB0zxMA%3D"
tile_names = [tile[-5:] for tile in adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token, anon=False).ls('snowmelt/eric/MGRS/')]

In [3]:
#tile_names

In [4]:
f'{(len(tile_names)/273)*100:.2f}% coverage of our areas of interest'

'96.70% coverage of our areas of interest'

In [ ]:
#import getpass
#import azure.storage.blob
#import io

#sas_token = getpass.getpass()  # prompts for the sas_token
#sas_token = "se=2023-02-20T20%3A51Z&sp=racwdl&sv=2018-11-09&sr=c&skoid=b4d39ccb-ca9c-4e9d-a183-5e0f3ba3e609&sktid=f6b6dd5b-f02f-441a-99a0-162ac5060bd2&skt=2023-02-13T20%3A52%3A06Z&ske=2023-02-20T20%3A51%3A00Z&sks=b&skv=2018-11-09&sig=C4rfs7%2BUt5kqABSyxX8qZ0R%2Bbuh9CMlRrYvRN1VtSwc%3D"

#container_client = azure.storage.blob.ContainerClient(
#    "https://snowmelt.blob.core.windows.net",
#    container_name="snowmelt",
#    credential=sas_token,
#)

In [ ]:
#file_list = [item['name'] for item in list(container_client.list_blobs())]
#url = "https://snowmelt.blob.core.windows.net/snowmelt/"

In [ ]:
#file_list

In [ ]:
resolution = 40
nodata = 'nan'

In [ ]:
#list(filter(lambda x: str({resolution}) in x, file_list))

In [ ]:
#for nm in tile_names:
#    list(filter(lambda x: nm in x, list(filter(lambda x: str(resolution) in x, file_list))))

In [ ]:
m = folium.Map(
    location=(40.76, -111.89),
    tiles="Stamen Terrain", #"cartodbpositron"
    zoom_start=7,
)

#m = folium.plugins.DualMap(
#    location=(40.76, -111.89),
#    tiles="Stamen Terrain", #"cartodbpositron"
#    zoom_start=7,
#)

minimap = plugins.MiniMap(toggle_display=True)
m.add_child(minimap)

interannual_medians = folium.FeatureGroup(name='2015-2022 Median Snowmelt Runoff Onset Date')
interannual_std = folium.FeatureGroup(name='2015-2022 Standard Deviation of Snowmelt Runoff Onset Date',show=False)
interannual_polyfit = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date Trend',show=False)
interannual_corr = folium.FeatureGroup(name='2015-2022 Snowmelt Runoff Onset Date Trend Strength',show=False)
year_2015 = folium.FeatureGroup(name='2015 Snowmelt Runoff Onset Date',show=False)
year_2016 = folium.FeatureGroup(name='2016 Snowmelt Runoff Onset Date',show=False)
year_2017 = folium.FeatureGroup(name='2017 Snowmelt Runoff Onset Date',show=False)
year_2018 = folium.FeatureGroup(name='2018 Snowmelt Runoff Onset Date',show=False)
year_2019 = folium.FeatureGroup(name='2019 Snowmelt Runoff Onset Date',show=False)
year_2020 = folium.FeatureGroup(name='2020 Snowmelt Runoff Onset Date',show=False)
year_2021 = folium.FeatureGroup(name='2021 Snowmelt Runoff Onset Date',show=False)
year_2022 = folium.FeatureGroup(name='2022 Snowmelt Runoff Onset Date',show=False)
layer_all_years = [year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022]
years = [2015,2016,2017,2018,2019,2020,2021,2022]

tiler = "https://api.cogeo.xyz/cog/tiles/{z}/{x}/{y}"
for tile in tile_names:
    tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_allyears_median_{resolution}m.tif"
    expression = f"rescale=0,365&colormap_name=viridis&nodata={nodata}"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_medians.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2015-2022 Median Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    
    tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_allyears_std_{resolution}m.tif"
    expression = f"rescale=0,90&colormap_name=reds&nodata={nodata}"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_std.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, opacity=1,name='2015-2022 STD Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))

    tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_allyears_polyfit_{resolution}m.tif"
    expression = f"rescale=-10,10&colormap_name=rdbu_r&nodata={nodata}"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_polyfit.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True, opacity=1,name='2015-2022 Snowmelt Runoff Onset Trend', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
 
    tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_allyears_corr_strength_{resolution}m.tif"
    expression = f"rescale=-1,1&colormap_name=rdbu_r&nodata={nodata}"
    virtual_tiles = f"{tiler}?url={tif}&{expression}"
    interannual_corr.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name='2015-2022 Snowmelt Runoff Onset Trend Strength', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))

    

for layer_year, year in zip(layer_all_years,years):
    for tile in tile_names:
        tif = f"https://snowmelt.blob.core.windows.net/snowmelt/eric/MGRS/{tile}/runoff_onset_{tile}_{year}_median_{resolution}m.tif"
        expression = f"rescale=0,365&colormap_name=viridis&nodata={nodata}"
        virtual_tiles = f"{tiler}?url={tif}&{expression}"
        layer_year.add_child(folium.TileLayer(tiles=virtual_tiles,overlay=True,opacity=1,name=f'{year} Snowmelt Runoff Onset', attr="Microsoft PC hosted Sentinel-1 RTC via Catalyst"))
    

interannual_medians.add_to(m)
interannual_std.add_to(m)
interannual_polyfit.add_to(m)
interannual_corr.add_to(m)

for layer_year in layer_all_years:
    layer_year.add_to(m)


folium.LayerControl().add_to(m)

cm = branca.colormap.linear.viridis.scale(0,365)
cm.add_to(m)

m

In [ ]:
m.save("index.html")